## Remote Training - Azure Machine Learning Service

In [2]:
import azureml.core
from azureml.core import Workspace

# print core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.18


### Connect to Workspace

In [3]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

Found the config file in: /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/config.json
ML-Workspace	westeurope	ML-DEV	westeurope


### Creating an Experiment

In [4]:
experiment_name = 'vehicle-views-25000-sessions'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Create or Attach existing compute resource

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. gpucluster


### Get and Upload data for training in Cloud

In [7]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=data_folder, target_path='datasets', overwrite=True, show_progress=True)

AzureBlob mlworkspace0237397596 azureml-blobstore-b63b2fa4-c464-45f5-b678-4f311b598992
Uploading /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/data/dataset_small_25000_sessions/all_train_seq.dat
Uploading /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/data/dataset_small_25000_sessions/test.dat
Uploading /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/data/dataset_small_25000_sessions/train.dat
Uploaded /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/data/dataset_small_25000_sessions/test.dat, 1 files out of an estimated total of 3
Uploaded /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/data/dataset_small_25000_sessions/train.dat, 2 files out of an estimated total of 3
Uploaded /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/data/dataset_small_25000_sessions/all_train_seq.dat, 3 files out of an estimated total of 3


$AZUREML_DATAREFERENCE_69eca7aed535465aab7d2c320f29ae2b

### Train on Remote Cluster